In [9]:
import numpy as np
import mujoco as mujoco
import matplotlib.pyplot as plt
import matplotlib.animation as animate
import jax.numpy as jnp

import mujoco_funcs as mj_funcs
import mjcf_models as mj_models
import visualize_mj_funcs as mj_vis

import dyn_functions as dyn
import visualize_dyn_funcs as vis_dyn
import gen_ctrl_funcs as gen_ctrl
import analyze_ilqr_output_funcs as analyze
np.set_printoptions(precision=7, suppress=True, linewidth=100)


In [15]:
dt = 0.01
len_seq = 200
time_vec = np.arange(0,len_seq*dt, dt)
h_bar = 1.0
r_bar = 0.05
m_bar = 1.0
moi = (1/12)*h_bar*(m_bar**2 + 3*r_bar**2)
x_init = np.array([0.0, 0.0])
u_lin = np.array([0.0]) 
x_seq = np.zeros((len_seq, len(x_init)))
x_seq[0]= x_init

In [16]:
mjcf_model  = mj_models.create_MJCF_pm_pend_dev()
mj_model, mj_render, mj_data = mj_funcs.create_mujoco_model(mjcf_model, dt)

In [17]:

nu = mj_model.nu
nx = mj_model.nv    
eps = 1e-6
A = np.zeros((2*nx, 2*nx))
B = np.zeros((2*nx,   nu))
flg_centered = False
mj_model.opt.timestep = dt
mj_data.ctrl = u_lin
mujoco.mj_forward(mj_model,mj_data)
print(mj_data.qpos)
print(mj_data.qvel)
print(mj_data.ctrl)

[0.]
[0.]
[0.]


In [18]:
pend_sys = dyn.single_pm_pend_dyn(g=9.81,b=0.0,l=h_bar)
disc_dyn_func = lambda x, u: gen_ctrl.step_rk4(pend_sys.cont_dyn_func, dt, x, u)
pend_cont_lin_sys = pend_sys.cont_lti_pend_ss_down()
pend_disc_lin_sys = gen_ctrl.discretize_continuous_state_space(pend_cont_lin_sys, dt, c2d_method='zohCombined')
print('dyn_lin_A: \n', pend_disc_lin_sys.a)
print('dyn_lin_B: \n', pend_disc_lin_sys.b)


dyn_lin_A: 
 [[ 0.99951  0.01   ]
 [-0.09808  0.99951]]
dyn_lin_B: 
 [[0.00005]
 [0.01   ]]


In [14]:
x_lin = np.array([0.0, 0.0])
u_lin = np.array([0.0])
mj_funcs.set_mj_ctrl_vec(mj_data, u_lin)
mj_funcs.set_mj_state_vec(mj_data, x_lin)
mujoco.mj_forward(mj_model, mj_data)
A_mj,  B_mj  = mj_funcs.linearize_mujoco_state_and_control(mj_model, mj_data, eps=1e-6, flg_centered=True)    
x_lin_jax = jnp.array(x_lin)
u_lin_jax = jnp.array(u_lin)
xu_lin_jax = jnp.concatenate((x_lin_jax, u_lin_jax))
A_dyn, B_dyn = gen_ctrl.linearize_dynamics(disc_dyn_func, xu_lin_jax, len(x_lin))



print('A mujoco: \n', A_mj)
print('A dyn: \n',    A_dyn)
print('B mujoco: \n', B_mj )
print('B dyn: \n',    B_dyn)

x_k = np.array([0.0,0.0])
u_k = np.array([0.1])

x_kp1_pred_mj = A_mj @ x_k.reshape(-1,1) + B_mj @ u_k.reshape(-1,1)
x_kp1_pred_dyn = A_dyn @ x_k.reshape(-1,1) + B_dyn @ u_k.reshape(-1,1)

mj_funcs.set_mj_ctrl_vec(mj_data, u_k)
mj_funcs.set_mj_state_vec(mj_data, x_k)
mujoco.mj_forward(mj_model, mj_data)
mujoco.mj_step(mj_model, mj_data)

x_kp1_mj = mj_funcs.get_state_vec(mj_data)


x_kp1_ivp = gen_ctrl.step_solve_ivp(pend_sys.cont_dyn_func, dt, x_k, u_k)

print('mj predicted: \n', x_kp1_pred_mj)
print('dyn predicted rk4: \n', x_kp1_pred_dyn)
print('mj simulated: \n', x_kp1_mj)
print('dyn simulated ivp: \n', x_kp1_ivp)

A mujoco: 
 [[ 0.97548  0.05   ]
 [-0.49045  1.     ]]
A dyn: 
 [[ 0.98776  0.0498 ]
 [-0.4885   0.98776]]
B mujoco: 
 [[0.0025]
 [0.05  ]]
B dyn: 
 [[0.00125]
 [0.0498 ]]
mj predicted: 
 [[0.00025]
 [0.005  ]]
dyn predicted rk4: 
 [[0.00012]
 [0.00498]]
mj simulated: 
 [0.00025 0.005  ]
dyn simulated ivp: 
 [[0.00012]
 [0.00498]]
